In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import gensim
from gensim.models import Word2Vec
from IPython.display import display,HTML
import tensorflow.compat.v1 as tf

import warnings
warnings.filterwarnings("ignore")

In [2]:
stemmer = PorterStemmer()

In [3]:
stopwords.words("bengali")

['অতএব',
 'অথচ',
 'অথবা',
 'অনুযায়ী',
 'অনেক',
 'অনেকে',
 'অনেকেই',
 'অন্তত',
 'অন্য',
 'অবধি',
 'অবশ্য',
 'অর্থাত',
 'আই',
 'আগামী',
 'আগে',
 'আগেই',
 'আছে',
 'আজ',
 'আদ্যভাগে',
 'আপনার',
 'আপনি',
 'আবার',
 'আমরা',
 'আমাকে',
 'আমাদের',
 'আমার',
 'আমি',
 'আর',
 'আরও',
 'ই',
 'ইত্যাদি',
 'ইহা',
 'উচিত',
 'উত্তর',
 'উনি',
 'উপর',
 'উপরে',
 'এ',
 'এঁদের',
 'এঁরা',
 'এই',
 'একই',
 'একটি',
 'একবার',
 'একে',
 'এক্',
 'এখন',
 'এখনও',
 'এখানে',
 'এখানেই',
 'এটা',
 'এটাই',
 'এটি',
 'এত',
 'এতটাই',
 'এতে',
 'এদের',
 'এব',
 'এবং',
 'এবার',
 'এমন',
 'এমনকী',
 'এমনি',
 'এর',
 'এরা',
 'এল',
 'এস',
 'এসে',
 'ঐ',
 'ও',
 'ওঁদের',
 'ওঁর',
 'ওঁরা',
 'ওই',
 'ওকে',
 'ওখানে',
 'ওদের',
 'ওর',
 'ওরা',
 'কখনও',
 'কত',
 'কবে',
 'কমনে',
 'কয়েক',
 'কয়েকটি',
 'করছে',
 'করছেন',
 'করতে',
 'করবে',
 'করবেন',
 'করলে',
 'করলেন',
 'করা',
 'করাই',
 'করায়',
 'করার',
 'করি',
 'করিতে',
 'করিয়া',
 'করিয়ে',
 'করে',
 'করেই',
 'করেছিলেন',
 'করেছে',
 'করেছেন',
 'করেন',
 'কাউকে',
 'কাছ',
 'কাছে',
 'কাজ',
 'কাজে',
 'কারও',
 '

In [4]:
list_len = stopwords.words("bengali")
len(list_len)

398

In [5]:
bengali_data = pd.read_csv("bengali_stopwords.txt")

In [6]:
bengali_data.head()

,অতএব
0,অথচ
1,অথবা
2,অনুযায়ী
3,অনেক
4,অনেকে


In [7]:
bengali_data.shape

(397, 1)

In [8]:
bengali_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   অতএব    397 non-null    object
dtypes: object(1)
memory usage: 3.2+ KB


In [9]:
bengali_data.isnull().sum()

অতএব    0
dtype: int64

In [10]:
paragraph = "রান্নায় ব্যবহৃত অন্যতম গুরুত্বপূর্ণ মশলা হল জিরা। বেশিরভাগ ভারতীয় রান্নাতেই এই মশলার ব্যবহার হয়ে থাকে। তবে জিরা কেবলমাত্র খাবারের স্বাদ-গন্ধ বাড়ায় না, উপরন্তু এটি আমাদের স্বাস্থ্যেরও অনেক উপকার করে। ওজন কমানো, রক্তচাপ নিয়ন্ত্রণ করা, হজমের সমস্যা দূর করা, ডায়াবেটিস নিয়ন্ত্রণ এবং রক্তাল্পতা নিরাময় এই ধরনের নানা স্বাস্থ্যকর দিকের খেয়াল রাখে জিরা। জিরা তামা, আয়রন, অ্যান্টিঅক্সিডেন্ট, ভিটামিন-এ, ভিটামিন-সি, জিঙ্ক এবং পটাশিয়ামের সমৃদ্ধ উৎস। বিভিন্ন ঘরোয়া প্রতিকারের ক্ষেত্রে জিরা ব্যবহার করা হয়ে থাকে। বিশেষ করে জিরার জল আমাদের স্বাস্থ্যের জন্য বিশেষ উপকারি। প্রতিদিন মাত্র এক গ্লাস জিরার জল পান করা আমাদের সামগ্রিক স্বাস্থ্যের জন্য ভীষণ উপকারি হতে পারে। বদহজম নিরাময় করে: অন্ত্রে গ্যাস জমে গিয়ে পেট ফুলে যাওয়ার ঘটনা খুবই সাধারণ ব্যাপার। এর ফলে, পেট ফুলে শক্ত হয়ে যায় এবং পেট ভার হয়ে থাকার অনুভূতি হয়। এছাড়াও, পেটে অস্বস্তি কিংবা পেট ব্যথাও হতে পারে। এক্ষেত্রে জিরা জলের পান করা অত্যন্ত উপকারি হতে পারে। জিরা বদহজমের সমস্যা থেকে বাঁচাতে পারে। ওজন কমাতে সাহায্য করে: জিরার জল ভাল হজমে সাহায্য করে এবং দেহ থেকে টক্সিন বার করতে পারে। পাচনতন্ত্র ঠিক থাকলে, শরীর থেকে টক্সিন সহজেই পরিষ্কার হয়ে যায়। যার ফলে ওজন আর ফ্যাট দুইই তাড়াতাড়ি কমে যায়। তাই নিয়মিত জিরার জল পান করলে তা ওজন নিয়ন্ত্রণে রাখতে পারে। রোগ প্রতিরোধ ক্ষমতা বৃদ্ধি করে: জিরা পটাশিয়াম, আয়রন এবং ফাইবারের একটি চমৎকার উৎস। নিয়মিত জিরার জলের সেবন, রোগ প্রতিরোধ ক্ষমতাকে শক্তিশালী করে তুলতে সহায়তা করে। এটি বিভিন্ন রোগের বিরুদ্ধে লড়াই করার পাশাপাশি, ঘনঘন অসুস্থ হওয়ার সম্ভাবনাকেও কম করতে সাহায্য করে। ডায়াবেটিস নিয়ন্ত্রণে রাখে: খালি পেটে জিরা জলের সেবন, ডায়াবেটিস আক্রান্ত রোগীদের ক্ষেত্রে অত্যন্ত উপকারি। জিরা শরীরের ইনসুলিন উৎপাদনকে উদ্দীপিত করে। এটি রক্তে শর্করার মাত্রা নিয়ন্ত্রণে রাখতে সহায়তা করে। ফলে ডায়াবেটিস নিয়ন্ত্রণে থাকে। রক্তচাপ নিয়ন্ত্রণ করে: জিরা জলে উচ্চ পটাশিয়াম থাকে। পটাশিয়াম একটি গুরুত্বপূর্ণ খনিজ, যা শরীরের সঠিক ক্রিয়াকলাপে গুরুত্বপূর্ণ ভূমিকা পালন করে। তাছাড়া, এটা নুনের ক্ষতিকারক প্রভাবগুলির মধ্যে ভারসাম্য বজায় রেখে রক্তচাপ নিয়ন্ত্রণে রাখতে সাহায্য করে।"

In [11]:
text_tokens = word_tokenize(paragraph)
tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
print(tokens_without_sw)

['রান্নায়', 'ব্যবহৃত', 'অন্যতম', 'গুরুত্বপূর্ণ', 'মশলা', 'জিরা।', 'বেশিরভাগ', 'ভারতীয়', 'রান্নাতেই', 'মশলার', 'হয়ে', 'থাকে।', 'জিরা', 'কেবলমাত্র', 'খাবারের', 'স্বাদ-গন্ধ', 'বাড়ায়', ',', 'উপরন্তু', 'স্বাস্থ্যেরও', 'উপকার', 'করে।', 'ওজন', 'কমানো', ',', 'রক্তচাপ', 'নিয়ন্ত্রণ', ',', 'হজমের', 'সমস্যা', 'দূর', ',', 'ডায়াবেটিস', 'নিয়ন্ত্রণ', 'রক্তাল্পতা', 'নিরাময়', 'ধরনের', 'স্বাস্থ্যকর', 'দিকের', 'খেয়াল', 'রাখে', 'জিরা।', 'জিরা', 'তামা', ',', 'আয়রন', ',', 'অ্যান্টিঅক্সিডেন্ট', ',', 'ভিটামিন-এ', ',', 'ভিটামিন-সি', ',', 'জিঙ্ক', 'পটাশিয়ামের', 'সমৃদ্ধ', 'উৎস।', 'ঘরোয়া', 'প্রতিকারের', 'জিরা', 'থাকে।', 'জিরার', 'জল', 'স্বাস্থ্যের', 'উপকারি।', 'প্রতিদিন', 'এক', 'গ্লাস', 'জিরার', 'জল', 'পান', 'সামগ্রিক', 'স্বাস্থ্যের', 'ভীষণ', 'উপকারি', 'পারে।', 'বদহজম', 'নিরাময়', ':', 'অন্ত্রে', 'গ্যাস', 'জমে', 'পেট', 'ফুলে', 'ঘটনা', 'খুবই', 'ব্যাপার।', ',', 'পেট', 'ফুলে', 'শক্ত', 'পেট', 'ভার', 'থাকার', 'অনুভূতি', 'হয়।', 'এছাড়াও', ',', 'পেটে', 'অস্বস্তি', 'পেট', 'ব্যথাও', 'পারে।', 'এক্ষেত্রে', 'জিরা', 'জলে

In [12]:
filtered_sentence = (" ").join(tokens_without_sw)
print(filtered_sentence)

রান্নায় ব্যবহৃত অন্যতম গুরুত্বপূর্ণ মশলা জিরা। বেশিরভাগ ভারতীয় রান্নাতেই মশলার হয়ে থাকে। জিরা কেবলমাত্র খাবারের স্বাদ-গন্ধ বাড়ায় , উপরন্তু স্বাস্থ্যেরও উপকার করে। ওজন কমানো , রক্তচাপ নিয়ন্ত্রণ , হজমের সমস্যা দূর , ডায়াবেটিস নিয়ন্ত্রণ রক্তাল্পতা নিরাময় ধরনের স্বাস্থ্যকর দিকের খেয়াল রাখে জিরা। জিরা তামা , আয়রন , অ্যান্টিঅক্সিডেন্ট , ভিটামিন-এ , ভিটামিন-সি , জিঙ্ক পটাশিয়ামের সমৃদ্ধ উৎস। ঘরোয়া প্রতিকারের জিরা থাকে। জিরার জল স্বাস্থ্যের উপকারি। প্রতিদিন এক গ্লাস জিরার জল পান সামগ্রিক স্বাস্থ্যের ভীষণ উপকারি পারে। বদহজম নিরাময় : অন্ত্রে গ্যাস জমে পেট ফুলে ঘটনা খুবই ব্যাপার। , পেট ফুলে শক্ত পেট ভার থাকার অনুভূতি হয়। এছাড়াও , পেটে অস্বস্তি পেট ব্যথাও পারে। এক্ষেত্রে জিরা জলের পান অত্যন্ত উপকারি পারে। জিরা বদহজমের সমস্যা বাঁচাতে পারে। ওজন কমাতে সাহায্য : জিরার জল ভাল হজমে সাহায্য দেহ টক্সিন পারে। পাচনতন্ত্র থাকলে , শরীর টক্সিন সহজেই পরিষ্কার হয়ে যায়। ওজন ফ্যাট দুইই তাড়াতাড়ি কমে যায়। নিয়মিত জিরার জল পান ওজন নিয়ন্ত্রণে রাখতে পারে। রোগ প্রতিরোধ ক্ষমতা বৃদ্ধি : জিরা পটাশিয়াম , আয়রন ফাইবা

In [13]:
paragraph

'রান্নায় ব্যবহৃত অন্যতম গুরুত্বপূর্ণ মশলা হল জিরা। বেশিরভাগ ভারতীয় রান্নাতেই এই মশলার ব্যবহার হয়ে থাকে। তবে জিরা কেবলমাত্র খাবারের স্বাদ-গন্ধ বাড়ায় না, উপরন্তু এটি আমাদের স্বাস্থ্যেরও অনেক উপকার করে। ওজন কমানো, রক্তচাপ নিয়ন্ত্রণ করা, হজমের সমস্যা দূর করা, ডায়াবেটিস নিয়ন্ত্রণ এবং রক্তাল্পতা নিরাময় এই ধরনের নানা স্বাস্থ্যকর দিকের খেয়াল রাখে জিরা। জিরা তামা, আয়রন, অ্যান্টিঅক্সিডেন্ট, ভিটামিন-এ, ভিটামিন-সি, জিঙ্ক এবং পটাশিয়ামের সমৃদ্ধ উৎস। বিভিন্ন ঘরোয়া প্রতিকারের ক্ষেত্রে জিরা ব্যবহার করা হয়ে থাকে। বিশেষ করে জিরার জল আমাদের স্বাস্থ্যের জন্য বিশেষ উপকারি। প্রতিদিন মাত্র এক গ্লাস জিরার জল পান করা আমাদের সামগ্রিক স্বাস্থ্যের জন্য ভীষণ উপকারি হতে পারে। বদহজম নিরাময় করে: অন্ত্রে গ্যাস জমে গিয়ে পেট ফুলে যাওয়ার ঘটনা খুবই সাধারণ ব্যাপার। এর ফলে, পেট ফুলে শক্ত হয়ে যায় এবং পেট ভার হয়ে থাকার অনুভূতি হয়। এছাড়াও, পেটে অস্বস্তি কিংবা পেট ব্যথাও হতে পারে। এক্ষেত্রে জিরা জলের পান করা অত্যন্ত উপকারি হতে পারে। জিরা বদহজমের সমস্যা থেকে বাঁচাতে পারে। ওজন কমাতে সাহায্য করে: জিরার জল ভাল হজমে সাহ

In [14]:
total_words = []
for text in filtered_sentence:
    for t in text.split():
        total_words.append(t)
        
total_words = set(total_words)

In [15]:
total_words

{',',
 '-',
 ':',
 '।',
 'ঁ',
 'অ',
 'আ',
 'ই',
 'উ',
 'এ',
 'ও',
 'ক',
 'খ',
 'গ',
 'ঘ',
 'ঙ',
 'চ',
 'ছ',
 'জ',
 'ট',
 'ঠ',
 'ড',
 'ণ',
 'ত',
 'থ',
 'দ',
 'ধ',
 'ন',
 'প',
 'ফ',
 'ব',
 'ভ',
 'ম',
 'য',
 'র',
 'ল',
 'শ',
 'ষ',
 'স',
 'হ',
 '়',
 'া',
 'ি',
 'ী',
 'ু',
 'ূ',
 'ৃ',
 'ে',
 'ো',
 '্',
 'ৎ',
 'ড়',
 'য়'}

In [16]:
#load Bengali text corpus
with open("bengali_story.txt", "r", encoding = "utf-8") as f:
    text = f.read()

In [17]:
#tokenize text
tokens = word_tokenize(text)

In [18]:
stop_words = set(stopwords.words("bengali"))

In [19]:
#remove stopwords
filtered_tokens = [word for word in tokens if not word in stop_words]

In [20]:
#build Word2Vec model
model = gensim.models.Word2Vec([filtered_tokens], min_count = 1)

In [21]:
#get most similar words to a given word
similar_words = model.wv.most_similar("কলকাতা")

In [22]:
print(similar_words)

[('নিদর্শন', 0.3420822024345398), ('পড়েছে', 0.3112014830112457), ('চংডীপাঠঃ', 0.27206185460090637), ('এখনো', 0.26115113496780396), ('বেজে', 0.2460700422525406), ('সরে', 0.23774054646492004), ('বললেন।', 0.2342851758003235), ('চল\u200c', 0.23392874002456665), ('সামনে।', 0.2335735261440277), ('বাড়িতে', 0.23286208510398865)]
